# LOGS: 
## N = 500
### Nelder-Mead : время работы 290 секунд, Gini: train 0.8, test 0.77
### differential-evolution : время работы порядка 15 минут, Gini: train 0.76, test 0.75 -- default params
### BFGS : время работы 95 секунд, Gini: train 0.8, test 0.76 -- default params
### CG : время работы 127 секунд, Gini: train 0.77, test 0.75 -- default params
### SLSQP : время работы 42 секунд, Gini: train 0.81, test 0.776 -- default params
### COBYLA : время работы 84 секунд, Gini: train 0.75, test 0.75 -- default params
### TNC : время работы 590 секунд, Gini: train 0.82, test 0.80 -- default params

## N = 200
### TNC 14 hyperplanes : время работы 834 секунд, Gini: train 0.85, test 0.83 -- default params

## N = 100
### TNC 21 hyperplanes : время работы 984 секунд, Gini: train 0.885, test 0.85 -- default params

## Сравнение с другими моделями:
### RandomForestClassifier: Gini: train 1.0, test 0.80 -- default params
### LGBMClassifier: Gini: train 0.998, test 0.86 -- default params
### CatBoostClassifier: Gini: train 0.997, test 0.89 -- default params

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy.optimize import minimize, differential_evolution
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel("Устойчивост эл_сети.xlsx",engine='openpyxl',sheet_name='Data_for_UCI_named')

In [3]:
df.tail()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
9995,2.930406,9.487627,2.376523,6.187797,3.343416,-0.658054,-1.449106,-1.236256,0.601709,0.779642,0.813512,0.608385,0.023892,unstable
9996,3.392299,1.274827,2.954947,6.894759,4.349512,-1.663661,-0.952437,-1.733414,0.502079,0.567242,0.285880,0.366120,-0.025803,stable
9997,2.364034,2.842030,8.776391,1.008906,4.299976,-1.380719,-0.943884,-1.975373,0.487838,0.986505,0.149286,0.145984,-0.031810,stable
9998,9.631511,3.994398,2.757071,7.821347,2.514755,-0.966330,-0.649915,-0.898510,0.365246,0.587558,0.889118,0.818391,0.037789,unstable
9999,6.530527,6.781790,4.349695,8.673138,3.492807,-1.390285,-1.532193,-0.570329,0.073056,0.505441,0.378761,0.942631,0.045263,unstable


In [4]:
d = {'unstable': 0, 'stable': 1}
df.stabf = df.stabf.map(d)

In [5]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,1
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,0


In [6]:
X = df.drop(['stab', 'stabf'], axis=1).values
y = df.stabf.values

In [7]:
for i in range(X.shape[1]):
    X[:,i]=(X[:,i]-X[:,i].min())/(X[:,i].max()-X[:,i].min())

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [55]:
tree = DecisionTreeClassifier(max_depth=7)
tree.fit(X_train, y_train)
train_preds = tree.predict(X_train)
test_preds = tree.predict(X_test)
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_preds) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_preds) - 1))

Train Gini = 0.7760174451729984
Test Gini = 0.6537581922529714


In [9]:
rf = RandomForestClassifier()

In [10]:
rf.fit(X_train, y_train)
train_preds = rf.predict(X_train)
test_preds = rf.predict(X_test)

In [11]:
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_preds) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_preds) - 1))

Train Gini = 1.0
Test Gini = 0.8029251258579901


In [30]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

In [31]:
lgbm.fit(X_train, y_train)
train_preds = lgbm.predict(X_train)
test_preds = lgbm.predict(X_test)

In [32]:
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_preds) - 1))
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_preds) - 1))

Train Gini = 0.9985429211742267
Test Gini = 0.8636193729678863


In [10]:
sc = Seniority_committee(N=100)

In [11]:
sc.fit(X_train, y_train, optim_method='TNC', maxiter=30, verbose=2)

Making hyperplane number 1
X_train.shape[0] = 7000
k = 7
L = 128
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -1.449142717602939
Time taken for optimization: 11.176649332046509
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -13.936313631860966
Time taken for optimization: 6.343890428543091
X_1.shape[0] = 195
X_0.shape[0] = 1297

Making hyperplane number 2
X_train.shape[0] = 5703
k = 7
L = 128
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -6.032262522274543
Time taken for optimization: 10.299799919128418
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.12302896282501027
Time taken for optimization: 5.888170003890991
X_1.shape[0] = 630
X_0.shape[0] = 21

Making hyperplane number 3


First approximation is obtained
The minimum of the loss function: -0.34340058832483755
Time taken for optimization: 7.429137945175171
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.33550163874125244
Time taken for optimization: 6.558955669403076
X_1.shape[0] = 50
X_0.shape[0] = 48
Cutted data shape is not enough

k = 5
L = 32
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.6005584088973951
Time taken for optimization: 6.724456071853638
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.6845051873272403
Time taken for optimization: 6.738415479660034
X_1.shape[0] = 63
X_0.shape[0] = 54
Cutted data shape is not enough

k = 4
L = 16
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -87.

The minimum of the loss function: -0.21199176273429118
Time taken for optimization: 5.6254191398620605
X_1.shape[0] = 16
X_0.shape[0] = 24
Cutted data shape is not enough

k = 5
L = 32
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.18382460944176288
Time taken for optimization: 5.620840311050415
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.7782635026798272
Time taken for optimization: 7.091850996017456
X_1.shape[0] = 22
X_0.shape[0] = 40
Cutted data shape is not enough

k = 4
L = 16
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.29606053375207875
Time taken for optimization: 6.465590000152588
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -1.4126548741139684
Time taken for

First approximation is obtained
The minimum of the loss function: -0.44100965878524556
Time taken for optimization: 4.161569356918335
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.2615108189348841
Time taken for optimization: 5.287962913513184
X_1.shape[0] = 65
X_0.shape[0] = 33
Cutted data shape is not enough

k = 3
L = 8
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -24.36911730932346
Time taken for optimization: 6.115485191345215
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -10.408625390923584
Time taken for optimization: 5.747735023498535
X_1.shape[0] = 45
X_0.shape[0] = 18
Cutted data shape is not enough

k = 2
L = 4
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -51.0535

First approximation is obtained
The minimum of the loss function: -0.27065808801752755
Time taken for optimization: 2.0961883068084717
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.06170049482877092
Time taken for optimization: 1.5711417198181152
X_1.shape[0] = 21
X_0.shape[0] = 8
Cutted data shape is not enough

k = 5
L = 32
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.24930960749496095
Time taken for optimization: 2.607900619506836
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -0.34870997055503683
Time taken for optimization: 2.0696468353271484
X_1.shape[0] = 32
X_0.shape[0] = 6
Cutted data shape is not enough

k = 4
L = 16
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -

First approximation is obtained
The minimum of the loss function: -36.02695444130003
Time taken for optimization: 2.965831756591797
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -12.827608427681334
Time taken for optimization: 2.520427703857422
X_1.shape[0] = 48
X_0.shape[0] = 13
Cutted data shape is not enough

k = 1
L = 2
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -48.00721406438399
Time taken for optimization: 4.017428874969482
Optimizing hyperplane for class 0
Optimization is started
First approximation is obtained
The minimum of the loss function: -33.821953363043754
Time taken for optimization: 3.5749194622039795
X_1.shape[0] = 90
X_0.shape[0] = 70
Cutted data shape is not enough

k = 0
L = 1
Optimizing hyperplane for class 1
Optimization is started
First approximation is obtained
The minimum of the loss function: -82.73341

In [12]:
sc.weights_hp # optim_weights = 

{1: (0,
  array([ 0.02347743,  0.03994415,  0.03912963,  0.0358237 , -0.07754966,
         -0.03582174, -0.02679344, -0.03031107,  0.00074428,  0.02622542,
          0.02523019,  0.02103864, -0.04242608]),
  0.007710100231303007,
  0.4469577415395406),
 2: (1,
  array([-0.01029176, -0.04451528, -0.02254985, -0.00280012,  0.02892913,
          0.00793972,  0.0090729 ,  0.01244328, -0.03594766, -0.01457453,
         -0.02816325, -0.03872933,  0.03748206]),
  0.9936507936507937,
  0.3790656416321703),
 3: (1,
  array([-0.02795467, -0.00328587, -0.03103596, -0.01494818,  0.04414138,
          0.01895   ,  0.00928745, -0.00021212, -0.00978925, -0.02566693,
         -0.01719733, -0.02184986,  0.01082632]),
  0.989010989010989,
  0.3563688407278675),
 4: (1,
  array([-0.09713729, -0.01018063,  0.0675843 , -0.05768072,  0.04567779,
          0.01095149,  0.01698898,  0.00918765,  0.00697072, -0.06239847,
         -0.05184121, -0.02097406, -0.00713352]),
  0.9739583333333334,
  0.33113428389018

In [87]:
# pd.DataFrame(optim_weights).to_pickle('weights_without_last_list.pickle')

In [13]:
sc_1 = Seniority_committee(N=100)

In [14]:
sc_1.weights_hp = sc.weights_hp

In [15]:
train_proba = sc_1.predict_proba(X_train)

Time taken to predict the targets: 5.277034044265747


In [16]:
test_proba = sc_1.predict_proba(X_test)

Time taken to predict the targets: 2.3339550495147705


In [17]:
print('Train Gini = {}'.format(2 * roc_auc_score(y_train, train_proba) - 1))

Train Gini = 0.8849357372714146


In [18]:
print('Test Gini = {}'.format(2 * roc_auc_score(y_test, test_proba) - 1))

Test Gini = 0.8473687817743731


In [19]:
train_preds = sc_1.predict(X_train)
test_preds = sc_1.predict(X_test)
print('Train F1 = {}'.format(f1_score(y_train, train_preds)))
print('Test F1 = {}'.format(f1_score(y_test, test_preds)))

Time taken to predict the targets: 6.230452537536621
Time taken to predict the targets: 2.4784576892852783
Train F1 = 0.8521551724137931
Test F1 = 0.8302277432712215


In [20]:
print('Train accuracy = {}'.format(accuracy_score(y_train, train_preds)))
print('Test accuracy = {}'.format(accuracy_score(y_test, test_preds)))

Train accuracy = 0.902
Test accuracy = 0.8906666666666667


In [9]:
class Seniority_committee:
    """
        ML Algo based on the seniority committee method
    """
    
    def __init__(self, N):
        """
            :param N: число наблюдений, находящихся выше гиперплоскости
        """
        
        self.X_train = None
        self.y_train = None
        self.L = -1
        self.N = N
        self.weights_hp = dict() # здесь будут записаны оптимальные веса гиперплоскостей в виде: 
                                 # {'num_hyperplane': (voted_class, optim_weights, probability)}
        self.optim_params = dict() # здесь будут записаны параметры оптимизации гиперплоскостей в виде:
                   # {'cycle_range': int, 'disp': bool, 'adaptive': bool, 'maxiter': int or None, 'xatol': float or None}
                   # значение параметров дано в описании метода make_hyperplane
        
    def expand(self, X):
        """
            Concatenates the feature matrix with the column of ones
            
            :param X: матрица признаков
            :returns: исходная матрица признаков со столбцом единиц (добавили bias)
        """
        X_new = X.copy()
        ones_col = np.ones(len(X_new))
        ones_col.shape = (len(X_new), 1)
        X_new = np.hstack((X_new, ones_col))
        return X_new
    
    
    def probability(self, X, w):
        """
            Принимает на вход матрицу фичей и вектор весов
            Возвращает предсказание вероятность того, что y = 1 при фиксированных x, P(y=1|x)

            :param X: матрица признаков 
            :param w: вектор весов
            :returns: вероятность того, что y = 1 при фиксированных x, P(y=1|x) ## вектор вероятностей = ReLU1(X.T*w)
        """
        
        linear = np.dot(X, w)
        linear[linear < 0] = 0
        linear[linear > 1] = 1
        
        return linear
    
    
    def compute_loss(self, X, y, w):
        """
            Принимает на вход матрицу весов, вектор ответов, вектор весов и параметр L, 
            влияющий на долю класса 0 в отсекающей гиперплоскости.
            Выдаёт на выход значение функции потерь
            
            :param X: матрица признаков
            :param w: вектор целевой переменной
            :param w: вектор весов
            :returns: значение функции потерь
        """
        
        p1 = self.probability(X, w)
        loss = np.sum((self.L - (self.L + 1) * y) * p1)
        
        return loss
    
    
    def compute_train_loss_class_0(self, w):
        """
            Function that we want to minimize, the committee member votes for class 0
            
            :param w: вектор весов
            :returns: значение функции потерь на обучающей выборке, когда член коммитета голосует за класс 0
        """
        
        if self.X_train is None or self.y_train is None:
            raise Exception('Model is not fitted')
        
        return self.compute_loss(self.X_train, 1 - self.y_train, w)
    
    
    def compute_train_loss_class_1(self, w):
        """
            Function that we want to minimize, the committee member votes for class 1
            
            :param w: вектор весов
            :returns: значение функции потерь на обучающей выборке, когда член коммитета голосует за класс 1
        """
        
        if self.X_train is None or self.y_train is None:
            raise Exception('Model is not fitted')
        
        return self.compute_loss(self.X_train, self.y_train, w)
    
    
    def make_hyperplane(self, class_num, X_train, optim_method, c=0.1, cycle_range=100, disp=False, \
                        adaptive=True, maxiter=None, xatol=None, verbose=0):
        """
            Function that makes one of three hyperplanes
            
            :param class_num: класс, за который голосует данный член комитета: [0, 1]
            :param X_train: матрица признаков обучающей выборки
            :param optim_method: метод оптимизации: ['Nelder-Mead', 'differential_evolution', 'BFGS', 'CG', 'SLSQP', \
                                                    'COBYLA', 'TNC']
            :param с: вспомогательный коэффициент для выбора начального приближения
            :param cycle_range: количество итераций минимизации функции потерь
            Параметры оптимизации с помощью алгоритма Нелдера-Мида:
                :param disp: bool: печать сообщения о сходимости
                :param adaptive: bool: адаптация параметров алгоритма для размерности задачи (полезно при больших размерностях)
                :param maxiter: максимально допустимое количество итераций при оптимизации
                :param xatol: абсолютная ошибка на оптимальных точках между итерациями, приемлемая для сходимости
            :param verbose: подробный вывод описания обучения: [0, 1, 2]:
                0 - не печатать ничего
                1 - печатать общее время обучения
                2 - подробный вывод информации о процессе обучения
            :returns: значение функции потерь на тестовой выборке
        """

        if self.X_train is None or self.y_train is None:
            raise Exception('Using make_hyperplane method before fitting')
        if class_num not in [0, 1]:
            raise Exception('Only binary classification is available, class_num should be 0 or 1')
        if optim_method not in ['Nelder-Mead', 'differential_evolution', 'BFGS', 'CG', 'SLSQP', \
                                                    'COBYLA', 'TNC']:
            raise Exception("Unavailable optimization method, only ['Nelder-Mead', 'differential_evolution', 'BFGS', 'CG', 'SLSQP', 'COBYLA', 'TNC'] are available")
        
        optim_result = []
        optim_result_more_precise = []
        start_time = time.time()
        
        if verbose == 2:
            print('Optimization is started')
        
        if optim_method == 'Nelder-Mead':
            
            for i in range(cycle_range):

                start_w = np.array((np.random.rand(X_train.shape[1]) - 0.5) * c)

                if class_num == 1:

                        res = minimize(self.compute_train_loss_class_1, x0=start_w, method='Nelder-Mead', \
                                       options={'disp': disp,'adaptive': adaptive, 'maxiter': maxiter, 'xatol': xatol})

                elif class_num == 0:

                        res = minimize(self.compute_train_loss_class_0, x0=start_w, method='Nelder-Mead', \
                                       options={'disp': disp,'adaptive': adaptive, 'maxiter': maxiter, 'xatol': xatol})

                optim_result.append([res.fun, res.x])


            if optim_result == []:
                raise Exception('We\'ve not get any satisfying first approximation')

            optim_result.sort(key=lambda x: x[0])
            optim_result = [x[1] for x in optim_result][:int(0.1 * cycle_range)]
            
            if verbose == 2:
                print('First approximation is obtained')

            for start_w_new in optim_result:

                if class_num == 1:
                    res = minimize(self.compute_train_loss_class_1, x0=start_w_new, method='Nelder-Mead', \
                             options={'disp': False, 'adaptive':True, 'xatol':1})
                elif class_num == 0:
                    res = minimize(self.compute_train_loss_class_0, x0=start_w_new, method='Nelder-Mead', \
                             options={'disp': False, 'adaptive':True, 'xatol':1})

                for k in range(2):
                    if class_num == 1:
                        res = minimize(self.compute_train_loss_class_1, x0=res.x, method='Nelder-Mead', \
                                 options={'disp': False, 'adaptive':True})
                    elif class_num == 0:
                        res = minimize(self.compute_train_loss_class_0, x0=res.x, method='Nelder-Mead', \
                                 options={'disp': False, 'adaptive':True})

                optim_result_more_precise += [[res.fun, res.x]]
                
            optim_result_more_precise = pd.DataFrame(optim_result_more_precise)
            optim_result_more_precise = optim_result_more_precise.sort_values(0).head(1)
            hyperplane_coefficients = optim_result_more_precise[1].values[0]
            min_loss_func = optim_result_more_precise[0].values[0]
        
        elif optim_method == 'differential_evolution':
            
            bounds = []
            for j in range(X_train.shape[1]):
                bounds.append((-10, 10))
                
            if class_num == 1:
                res = differential_evolution(self.compute_train_loss_class_1, bounds=bounds)
            
            if class_num == 0:
                res = differential_evolution(self.compute_train_loss_class_0, bounds=bounds)
            
            hyperplane_coefficients = res.x
            min_loss_func = res.fun
            
        else:
            
            for i in range(cycle_range):

                start_w = np.array((np.random.rand(X_train.shape[1]) - 0.5) * c)

                if class_num == 1:

                        res = minimize(self.compute_train_loss_class_1, x0=start_w, method=optim_method)

                elif class_num == 0:

                        res = minimize(self.compute_train_loss_class_0, x0=start_w, method=optim_method)

                optim_result.append([res.fun, res.x])


            if optim_result == []:
                raise Exception('We\'ve not get any satisfying first approximation')

            optim_result.sort(key=lambda x: x[0])
            optim_result = [x[1] for x in optim_result][:int(0.1 * cycle_range)]
            
            if verbose == 2:
                print('First approximation is obtained')

            for start_w_new in optim_result:

                if class_num == 1:
                    res = minimize(self.compute_train_loss_class_1, x0=start_w_new, method=optim_method)
                elif class_num == 0:
                    res = minimize(self.compute_train_loss_class_0, x0=start_w_new, method=optim_method)

                for k in range(2):
                    if class_num == 1:
                        res = minimize(self.compute_train_loss_class_1, x0=res.x, method=optim_method)
                    elif class_num == 0:
                        res = minimize(self.compute_train_loss_class_0, x0=res.x, method=optim_method)

                optim_result_more_precise += [[res.fun, res.x]]
                
            optim_result_more_precise = pd.DataFrame(optim_result_more_precise)
            optim_result_more_precise = optim_result_more_precise.sort_values(0).head(1)
            hyperplane_coefficients = optim_result_more_precise[1].values[0]
            min_loss_func = optim_result_more_precise[0].values[0]
        
        if verbose == 2:
            print('The minimum of the loss function: {0}'.format(min_loss_func))
        if verbose == 2:
            print('Time taken for optimization: {0}'.format(time.time() - start_time))

        return hyperplane_coefficients
        
    
    def cutter(self, X, w):
        """
            Function that makes binary targets for rational numbers
            
            :param X: матрица признаков
            :param w: вектор оптимальных весов
            :returns: бинаризованные предсказания целевой переменной
        """
        
        linear = np.dot(X, w)
        linear[linear < 0] = 0
        
        return np.sign(linear)
    
    
    def fit(self, X, y, optim_method='Nelder-Mead', cycle_range=100, disp=False, adaptive=True, maxiter=10, xatol=0.3, \
           verbose=0):
        """
            Fits the algorithm on the train sample 
            
            :param X: матрица признаков, обучающая выборка
            :param y: вектор истинных значений целевой переменной обучающей выборки
            :param optim_method: метод оптимизации: ['Nelder-Mead', 'differential_evolution', 'BFGS', 'CG', 'SLSQP', \
                                                    'COBYLA', 'TNC']
            Параметры оптимизации с помощью алгоритма Нелдера-Мида:
                :param disp: bool: печать сообщения о сходимости
                :param adaptive: bool: адаптация параметров алгоритма для размерности задачи (полезно при больших размерностях)
                :param maxiter: максимально допустимое количество итераций при оптимизации
                :param xatol: абсолютная ошибка на оптимальных точках между итерациями, приемлемая для сходимости
            :param verbose: подробный вывод описания обучения: [0, 1, 2]:
                0 - не печатать ничего
                1 - печатать общее время обучения
                2 - подробный вывод информации о процессе обучения
            :returns: -, но на выходе обученная моделька
        """
        
        start_of_fit_time = time.time()
        
        X_new = self.expand(X)
        
        self.X_train = X_new
        self.y_train = y
        self.optim_params = {'cycle_range': cycle_range, 'disp': disp, 'adaptive': adaptive, \
                             'maxiter': maxiter, 'xatol': xatol}
        
        hp_num = 1
        
        while self.X_train.shape[0] > 2 * self.N:
            
            if verbose == 2:
                print('Making hyperplane number {}'.format(hp_num))
                print('X_train.shape[0] = {}'.format(self.X_train.shape[0]))
            
            k = 7
            
            while True:
                
                if verbose == 2:
                    print('k = {}'.format(k))
                
                self.L = 2 ** k
                
                if verbose == 2:
                    print('L = {}'.format(self.L))
                
                if verbose == 2:
                    print('Optimizing hyperplane for class 1')
                
                hp_weights_class_1 = self.make_hyperplane(class_num=1, X_train=self.X_train, \
                                                     optim_method=optim_method, \
                                                     c=0.1, cycle_range=self.optim_params['cycle_range'], \
                                                     disp=self.optim_params['disp'], \
                                                     adaptive=self.optim_params['adaptive'], \
                                                     maxiter=self.optim_params['maxiter'], \
                                                     xatol=self.optim_params['xatol'], \
                                                     verbose=verbose)
                
                if verbose == 2:
                    print('Optimizing hyperplane for class 0')
                
                hp_weights_class_0 = self.make_hyperplane(class_num=0, X_train=self.X_train, \
                                                     optim_method=optim_method, \
                                                     c=0.1, cycle_range=self.optim_params['cycle_range'], \
                                                     disp=self.optim_params['disp'], \
                                                     adaptive=self.optim_params['adaptive'], \
                                                     maxiter=self.optim_params['maxiter'], \
                                                     xatol=self.optim_params['xatol'], \
                                                     verbose=verbose)
                
                cut_1 = self.cutter(self.X_train, hp_weights_class_1)
                cut_0 = self.cutter(self.X_train, hp_weights_class_0)
                X_1 = self.X_train[cut_1 == 1]
                y_1 = self.y_train[cut_1 == 1]
                y_1_rest = self.y_train[cut_1 == 0] # оставшиеся после отсечения сэмплы
                X_0 = self.X_train[cut_0 == 1]
                y_0 = self.y_train[cut_0 == 1]
                y_0_rest = self.y_train[cut_0 == 0] # оставшиеся после отсечения сэмплы
                
                if verbose == 2:
                    print('X_1.shape[0] = {}'.format(X_1.shape[0]))
                    print('X_0.shape[0] = {}'.format(X_0.shape[0]))
                
                if X_1.shape[0] < self.N and X_0.shape[0] < self.N:
                    
                    if verbose == 2:
                        print('Cutted data shape is not enough\n')
                    k -= 1
                    continue
                    
                else:
                    
                    if X_1.shape[0] >= X_0.shape[0]:
                        
                        proba = y_1.sum() / len(y_1)
                        proba_rest = y_1_rest.sum() / len(y_1_rest)
                        self.weights_hp[hp_num] = (1, hp_weights_class_1, proba, proba_rest)
                        
                        self.X_train = self.X_train[cut_1 == 0]
                        self.y_train = self.y_train[cut_1 == 0]
                        
                    elif X_1.shape[0] < X_0.shape[0]:
                        
                        proba = y_0.sum() / len(y_0)
                        proba_rest = y_0_rest.sum() / len(y_0_rest)
                        self.weights_hp[hp_num] = (0, hp_weights_class_0, proba, proba_rest)
                        
                        self.X_train = self.X_train[cut_0 == 0]
                        self.y_train = self.y_train[cut_0 == 0]
                        
                    hp_num += 1
                    if verbose == 2:
                        print()
                    break
                    
        end_of_fit_time = time.time()
        
        if verbose == 2 or verbose == 1:
            print('Time taken to fit the model: {0}'.format(end_of_fit_time - start_of_fit_time))
                

    def predict_proba(self, X):
        """
            Makes predict_proba for the sample
            
            :param X: матрица признаков, обучающая выборка
            :returns: предсказания вероятностей отнесения к классу 1
        """
        
        start_of_predict_time = time.time()
        
        X_new = self.expand(X)
        
        test_ = X_new.copy()
        
        if self.weights_hp == {}:
            raise Exception('Model is not fitted')
        
        predictions = pd.DataFrame({'proba': np.zeros(X.shape[0]), 'is_scored': False})
        
        for hp_num in self.weights_hp.keys():
            
            class_num, weights, proba, proba_rest = self.weights_hp[hp_num]
            
            cut = self.cutter(test_, weights)
            
            X_ = test_[cut == 1]
            test_ = test_[cut == 0]
            
            k = 0
            for i in range(predictions.shape[0]):
                
                if predictions.loc[i, 'is_scored'] == False:
                    if cut[k] == True:
                        predictions.loc[i, 'proba'] = proba
                        predictions.loc[i, 'is_scored'] = True
                    k += 1
            if hp_num == list(self.weights_hp.keys())[-1]:
                for i in range(predictions.shape[0]):
                    if predictions.loc[i, 'is_scored'] == False:
                        predictions.loc[i, 'proba'] = proba_rest
                        predictions.loc[i, 'is_scored'] = True
                        
        end_of_predict_time = time.time()
        print('Time taken to predict the targets: {0}'.format(end_of_predict_time - start_of_predict_time))
        
        return predictions['proba'].values
    
    def predict(self, X):
        """
            Makes predict_proba for the sample
            
            :param X: матрица признаков, обучающая выборка
            :returns: предсказания классов
        """
        
        proba = self.predict_proba(X)
        return np.array([round(x) for x in proba])